In [1]:
import os
import pandas as pd
from functions import get_all_origins, find_pattern_for_quantity, convert_to_grams, relation_qnt_preco, remove_spaces, clean_text
import re
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
df = get_all_origins()

In [8]:
# df[df['titulo'].str.contains(r'\bwhey\b', case=False, regex=True)]['titulo']

In [10]:
sabores_whey = 

['morango com chantilly',
 'pao de mel',
 'cheesecake de morango',
 'cocco cioccolato',
 'dulce de leche premium',
 'torta de banana',
 'strawberry milkshake',
 'banana cream',
 'torta al limone',
 'vanilla cream',
 'torta de limão',
 'cappuccino',
 'vitamina de frutas',
 'limão',
 'cheesecake de maracuja',
 'chocolate',
 'napolitano',
 'original',
 'torta cioccolato bianco',
 'original cafe',
 'doce de leite',
 'maracuja mousse',
 'beijinho',
 'double chocolate',
 'leite de coco',
 'cookies',
 'chocolate crocante',
 'brownie chocolate',
 'cocciocioccolato',
 'milho verde',
 'caramelo crocante',
 'duo bianco al latte',
 'strawberry milk shake',
 'churros',
 'cookies e cream',
 'cheesecake chocolate',
 'leite cacau avela',
 'beijinho de coco',
 'baunilha',
 'morango',
 'peanut butter',
 'cookies cream',
 'brigadeiro']

In [ ]:

df = df[df['ref'] == "7a68879e"]
pattern = r'(\d+[.,]?\d*)\s*(kg|g|gr|gramas)(?!\s*gratis)\b'
df[['quantidade', 'unidade']] = df['nome'].apply(lambda text: find_pattern_for_quantity(text, pattern)).apply(pd.Series)
df[['quantidade', 'unidade', 'titulo']]
df['qnt_gramas'] = df[['quantidade', 'unidade']].apply(convert_to_grams, axis=1)
df['preco_qnt'] = df.apply(relation_qnt_preco, axis=1)
df['qnt_gramas'] = df['qnt_gramas'].replace(-1, np.nan, inplace=True)
df

# pattern = r'(\d+)\s*(caps|cap|cáps|v-caps|cápsulas|capsules|cáp|comprimidos|comps|comp|capsulas|barras|soft|softgel|sachês|doses|dose|tabs|tabletes|tablets|gotas)\b'
# df[['quantidade_formato', 'formato']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)
# df['formato'] = df['formato'].apply(substituir_por_comprimidos)

# pattern = r'(\d+)\s*(caixa|display\s*c/|unidades|unid|un|saches|caixas|x)\b'
# df[['quantidade_formato_2', 'formato_2']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)